In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
# import os

# if not os.path.exists('./data'):
#     os.mkdir('./data')

# !wget --no-check-certificate \
#     https://storage.googleapis.com/laurencemoroney-blog.appspot.com/Sunspots.csv \
#     -O ./data/sunspots.csv

In [5]:
TRAIN_VAL_SPLIT = 3000

df_data = pd.read_csv('./data/sunspots.csv', index_col=0, skiprows=1, names=['ds', 'y'])
# df_data.plot()

df_train = df_data[:TRAIN_VAL_SPLIT]
df_val = df_data[TRAIN_VAL_SPLIT:]

In [7]:
WINDOW_SIZE = 64
BATCH_SIZE = 256
SHUFFER_BUFFER_SIZE = 1000

In [8]:
def build_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [9]:
dataset_train = build_dataset(df_train['y'].values, WINDOW_SIZE, BATCH_SIZE, SHUFFER_BUFFER_SIZE)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Lambda


def build_model():
    model = Sequential([
        Conv1D(filters=32, kernel_size=5, strides=1, padding="causal", activation="relu", input_shape=[None, 1]),
        LSTM(64, return_sequences=True),
        LSTM(64, return_sequences=True),
        Dense(30, activation="relu"),
        Dense(10, activation="relu"),
        Dense(1),
        Lambda(lambda x: x * 400)
    ])
    return model

In [15]:
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler
from util import clear_env

clear_env()

model = build_model()
model.compile(loss=Huber(), optimizer=SGD(lr=1e-8, momentum=0.9), metrics=['mae'])

def learning_rate(epoch):
    return 1e-8 * 10**(epoch / 20)

lr_scheduler = LearningRateScheduler(learning_rate)

history = model.fit(dataset_train, epochs=100, callbacks=[lr_scheduler], verbose=1)

Epoch 1/100


W0807 15:10:54.443367 4618917312 deprecation.py:323] From /Users/nex3z/dev/miniconda3/envs/tf2/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


12/12 [==============================] - 11s 886ms/step - loss: 64.4204 - mae: 64.6414
Epoch 2/100
12/12 [==============================] - 4s 335ms/step - loss: 62.0866 - mae: 62.3568
Epoch 3/100
12/12 [==============================] - 4s 339ms/step - loss: 58.8495 - mae: 59.1336
Epoch 4/100
12/12 [==============================] - 5s 380ms/step - loss: 55.2622 - mae: 55.5551
Epoch 5/100
 7/12 [================>.............] - ETA: 2s - loss: 48.6627 - mae: 49.1595

KeyboardInterrupt: 

In [ ]:
plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-8, 1e-4, 0, 60])